In [3]:
import pandas as pd
import numpy as np

### Загрузка датасета TOXRIC

Датасет TOXRIC состоит из двух частей, отличающихся способом введения хим. соединения в организм (intravenous/intravenous).
Скачаем обе части датасета из БД TOXRIC

In [4]:
# Примечание! Для того, чтобы скачать файл по ссылке нужно сначла открыть ссылку в браузере
df_intraperitoneal = pd.read_csv('https://toxric.bioinforai.tech/jk/DownloadController/DownloadToxicityInfo?toxicityId=44', delimiter=',')

In [6]:
# Примечание! Для того, чтобы скачать файл по ссылке нужно сначла открыть ссылку в браузере
df_intravenous = pd.read_csv('https://toxric.bioinforai.tech/jk/DownloadController/DownloadToxicityInfo?toxicityId=51', delimiter=',')

Теперь объединим оба датасета в один. При этом добавим новый столбец с указанием способа введения хим. соединения в организм (intravenous/intravenous). Пусть для способа intravenous значение будет равно 0, а для ntravenous - 1

In [7]:
df_intraperitoneal['Injection Method'] = 0
df_intravenous['Injection Method'] = 1
df_toxric = pd.concat([df_intraperitoneal, df_intravenous], axis=0)

In [8]:
df_toxric.shape

(51797, 8)

In [43]:
# Видим, что все наиболее значимые столбцы данных не содержат пропусков
df_toxric.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51797 entries, 0 to 16497
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TAID              51797 non-null  object 
 1   Name              6043 non-null   object 
 2   IUPAC Name        49449 non-null  object 
 3   PubChem CID       51797 non-null  int64  
 4   Smiles            51797 non-null  object 
 5   InChIKey          51746 non-null  object 
 6   Toxicity Value    51797 non-null  float64
 7   Injection Method  51797 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 3.6+ MB


In [13]:
# Проверим, есть ли в датасете полные дубли
duplicates = df_toxric.duplicated()
duplicated_rows = df_toxric[duplicates]
print(duplicates.sum()) # видно, что в датасете нет дублей
# Удалим их
df_toxric.drop_duplicates(inplace=True)

0


### Загрузка датасета ChEMBL

In [36]:
# Скачаем предобработанный датасет ChEMBL (расположен локально в репозитории)
df_chembl = pd.read_csv('../data/raw/ChEMBL_LD50_preprocessed.csv', delimiter=',')

In [42]:
df_chembl.shape

(6554, 9)

In [40]:
# Видно, что в датасете ChEMBL разные соединения тестировались на различных организмах, в то время как в датасете TOXRIC - только на мышах
df_chembl['Target Organism'].value_counts()

Target Organism
Mus musculus                4659
Rattus norvegicus            803
Helicoverpa assulta           78
Musca domestica               45
Chilo suppressalis            39
Plutella xylostella           36
Laodelphax striatellus        33
Nephotettix cincticeps        27
Cavia porcellus               27
Lucilia cuprina               21
Bombyx mori                   14
Mus sp.                       13
Oryctolagus cuniculus         11
Helicoverpa armigera           8
Canis lupus familiaris         6
Sitophilus zeamais             5
Gallus gallus                  5
Homo sapiens                   3
Macaca mulatta                 2
Ostrinia furnacalis            2
Spodoptera litura              2
Coturnix japonica              1
Anas platyrhynchos             1
Cercopithecidae                1
Plasmodium yoelii yoelii       1
Name: count, dtype: int64

### Объединим датафреймы ChEMBL и TOXRIC в один по столбцу SMILES

In [14]:
# Переименуем столбец Canonical SMILES в SMILES, чтобы объединить оба датафрейма по этому столбцу
df_toxric.rename(columns={'Canonical SMILES': 'Smiles'}, inplace=True)

In [22]:
merged_df = pd.merge(df_toxric, df_chembl, on='Smiles', how='left')

In [23]:
# Посмотрим на размеры исходных датасетов и датасета, полученного в результате слияния по столбцу SMILES
print(df_toxric.shape)
print(df_chembl.shape)
print(merged_df.shape)
# Видим, что часть строк присутствовала в обоих датасетах, поэтому результирующий датасет увеличился всего на 900 строк

(51797, 8)
(13758, 9)
(52664, 16)


Одной из проблем является то, что многие соединения, присутствующие в обоих датасетах, имеют разное значение LD50

In [44]:
# Выберем строки датафрейма, которые содержат значниия LD50, как для БД ChEMBL, так и для БД TOXRIC
# Отфильтруем строки, не содержащие значения LD50: 
# Для датасета ChEMBL это столбец Standard Value
# Для датасета TOXRIC - столбец Toxicity Value
tox_units = merged_df[pd.notna(merged_df['Toxicity Value']) & pd.notna(merged_df['Standard Value'])]
# В датасете ChEMBL значения LD50 представлены в различных единицах. 

# Выберем только те строки, где действие соединения оценивалось на мышах, как в датасете TOXRIС
tox_units = tox_units[tox_units['Target Organism'] == 'Mus musculus']\
    [['Toxicity Value', 'Injection Method', 'Standard Value', 'Standard Relation', 'Standard Units']]

In [45]:
# Посмотрим на распределение соединений по способу ввода в организм
tox_units['Injection Method'].value_counts()

Injection Method
0    868
1    442
Name: count, dtype: int64

In [46]:
# Выберем строки, соответствующие соединениями, вводимым с применением метода intraperitoneal
intraperitoneal = tox_units[tox_units['Injection Method'] == 0]

In [47]:
# Отфильтруем строки, в которых разница между значениями токсичности для одного и того же соединения составляет менее 10% 
intraperitoneal[abs(intraperitoneal['Toxicity Value'] - intraperitoneal['Standard Value']) < \
                intraperitoneal[['Toxicity Value','Standard Value']].min(axis=1) * 0.1]

,Toxicity Value,Injection Method,Standard Value,Standard Relation,Standard Units
8,73.299512,0,80.0,'=',mg.kg-1
191,66.996726,0,70.0,'=',mg.kg-1
251,55.648207,0,56.0,'=',mg.kg-1
285,319.992244,0,336.0,'=',mg.kg-1
430,999.977521,0,1000.0,'=',mg.kg-1
...,...,...,...,...,...
34955,42.994633,0,43.0,'=',mg.kg-1
34961,71.995506,0,72.0,'=',mg.kg-1
35461,799.973502,0,800.0,'>',mg.kg-1
35464,799.986514,0,800.0,'>',mg.kg-1


Видно, что из 868 соединений только 463 имеют значения токсичности (по данным из двух БД) отличающиеся не более чем на 10%

In [48]:
# Выберем строки, соответствующие соединениями, вводимым с применением метода intravenous
intravenous = tox_units[tox_units['Injection Method'] == 1]

In [49]:
# Отфильтруем строки, в которых разница между значениями токсичности для одного и того же соединения составляет менее 10% 
intravenous[abs(intravenous['Toxicity Value'] - intravenous['Standard Value']) < \
               intravenous[['Toxicity Value','Standard Value']].min(axis=1) * 0.1]

,Toxicity Value,Injection Method,Standard Value,Standard Relation,Standard Units
35845,79.999467,1,80.0,'=',mg.kg-1
35930,557.007800,1,609.0,'=',mg.kg-1
35963,179.995099,1,180.0,'=',mg.kg-1
36028,6.847763,1,7.0,'=',mg.kg-1
36174,93.999022,1,94.0,'=',mg.kg-1
...,...,...,...,...,...
52420,75.002488,1,75.0,'=',mg.kg-1
52421,63.006969,1,63.0,'=',mg.kg-1
52422,11.000932,1,11.0,'=',mg.kg-1
52423,6.000176,1,6.0,'=',mg.kg-1


Видно, что из 442 соединений только 224 имеют значения токсичности (по данным из двух БД) отличающиеся не более чем на 10%